In [ ]:
#!pip install sentinelhub

In [ ]:
#!printenv |sort

This notebook show how to use the odc_sh engine in order to load the Sentinel data using SentinelHUB.

In [ ]:
import os 
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
from sentinelhub import DataCollection
import pandas

%matplotlib inline

from odc_sh import engine

In [ ]:
sh_client_id=""
sh_client_secret=""

if not sh_client_id:
    sh_client_id = %env SH_CLIENT_ID

if not sh_client_secret:
    sh_client_secret = %env  SH_CLIENT_SECRET

Load data from Sentinel HUB. For other collections check the DataCollection or examples folder.

List all available collections (in some cases additional filter is needed in order to differenciate between the collection ids - e. g. DataCollection.SENTINEL1)

Initialize datacube engine

In [ ]:
dc = engine.Datacube(sh_client_id=sh_client_id, sh_client_secret=sh_client_secret)

In [ ]:
products = dc.list_sh_products()
#products

In [ ]:
bands = dc.list_sh_bands()
#bands

In [ ]:
col = dc.get_collection("SENTINEL2_L1C")
#col.bands

Load Sentinel 1 IW data. Additional filter with polarization needs to be specified. You can also limit the data by orbit.

In [ ]:
resolution = 20  # in meters
longitude = (11.987527, 12.004914)
latitude = (41.990797, 42.018348)

time = ("2019-01-01", "2019-01-04")

ds = dc.load(
    product=DataCollection.SENTINEL2_L1C,
    latitude=latitude,
    longitude=longitude,
    time=time,
    sh_resolution=resolution,
)

Display properties of the x-array datacube

Store downloaded data to .nc format

In [ ]:
#ds.to_netcdf("s2_all_bands.nc")

You can load the data 

In [ ]:
#ds_stored = xr.open_dataset("s2_all_bands.nc")
#ds_stored

Dislay available time intervals

In [ ]:
for t in ds.time:
    print(f"times: {t.data}")

Get first available frame

In [ ]:
crs = ccrs.UTM("18")
fig, ax = plt.subplots(nrows=1, ncols=len(ds.time), figsize=(12, 4), dpi=80)

for idx, t in enumerate(ds.time):
    print(f"time: {t.data}")
    single_time_frame = ds.sel(time=ds.time[idx], bands=['B04','B03','B02'], drop=True)
    single_time_frame= single_time_frame / single_time_frame.data.max()
    imgplot = single_time_frame.plot.imshow(ax=ax[idx], x="lon", y="lat", rgb="bands")
    imgplot.set_cmap('OrRd')
    ax[idx].axis('off')
plt.show()